In [2]:
import pandas as pd
#here's the download link for the people database: https://download.companieshouse.gov.uk/en_pscdata.html 
#use the file path from that dataset saved 

people_txt_path = "/home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc-snapshot-2025-10-14_1of30 (1)/psc-snapshot-2025-10-14_1of30.txt"
# Try newline-delimited JSON first (common for CH PSC snapshot), fallback to TSV/CSV
try:
	people_df = pd.read_json(people_txt_path, lines=True)
except ValueError:
	try:
		people_df = pd.read_csv(people_txt_path, sep="\t")
	except Exception:
		people_df = pd.read_csv(people_txt_path)

people_df.head()
print(people_df[1:10])

  company_number                                               data
1       08810260  {'address': {'address_line_1': 'Godmans Lane',...
2       06022970  {'address': {'address_line_1': '20 Rugby Road ...
3       08301480  {'address': {'address_line_1': 'Station Road',...
4       07428111  {'address': {'address_line_1': 'Cottage', 'add...
5       07425821  {'address': {'address_line_1': '66 Church Stre...
6       07874121  {'address': {'address_line_1': 'The Avenue Nor...
7       10523951  {'address': {'address_line_1': 'Priory Road', ...
8       10518671  {'address': {'address_line_1': 'Warneford Stre...
9       04333110  {'address': {'address_line_1': 'Gedney Marsh',...


In [4]:
# As a Python list
#print(people_df.columns.tolist)
#import pandas as pd
pd.set_option('display.max_columns', None)
display(pd.DataFrame({'column': people_df.columns}))

,column
0,company_number
1,data


In [ ]:
import json
import pandas as pd

# Pick first non-null row in `data`
if 'data' not in people_df.columns:
    raise KeyError("Column 'data' not found in people_df")

first_idx = people_df['data'].first_valid_index()
if first_idx is None:
    print("No non-null rows in people_df['data'].")
else:
    raw_val = people_df.loc[first_idx, 'data']

    # Parse if the cell is JSON text; otherwise assume it's already a dict
    if isinstance(raw_val, str):
        parsed_first = json.loads(raw_val)
    else:
        parsed_first = raw_val

    # 1) Top-level keys ("subcolumns") inside `data`
    print("Top-level keys in data:", sorted(parsed_first.keys()))

    # 2) Entire row of `data` (as a flattened table for readability)
    flat_first = pd.json_normalize(parsed_first, sep='.')
    display(flat_first.T)  # transpose to list fields vertically

    # 3) Optional: union of top-level keys across a small sample

Top-level keys in data: ['address', 'etag', 'identification', 'kind', 'links', 'name', 'natures_of_control', 'notified_on']


,0
etag,706b6e8773700f85361929bb5cd0214d630de6e0
kind,corporate-entity-person-with-significant-control
name,Robert Hitchins Limited
natures_of_control,"[ownership-of-shares-75-to-100-percent, voting..."
notified_on,2016-04-06
address.address_line_1,Boddington Lane
address.address_line_2,Boddington
address.country,England
address.locality,Cheltenham
address.postal_code,GL51 0TJ


Union of top-level keys in sample: ['address', 'ceased_on', 'country_of_residence', 'date_of_birth', 'etag', 'identification', 'kind', 'links', 'name', 'name_elements', 'nationality', 'natures_of_control', 'notified_on']


In [8]:
sample = people_df['data'].dropna().head(1000)
def to_dict(x):
    if isinstance(x, str):
        try:
            return json.loads(x)
        except Exception:
            return None
    return x if isinstance(x, dict) else None

sample_dicts = [to_dict(v) for v in sample]
sample_dicts = [d for d in sample_dicts if isinstance(d, dict)]
union_keys = sorted({k for d in sample_dicts for k in d.keys()})
print("Union of top-level keys in sample:", union_keys)

Union of top-level keys in sample: ['address', 'ceased_on', 'country_of_residence', 'date_of_birth', 'etag', 'identification', 'kind', 'links', 'name', 'name_elements', 'nationality', 'natures_of_control', 'notified_on']


In [11]:
df2 = {"company_number":"04333110","data":{"address":{"address_line_1":"Gedney Marsh","address_line_2":"Holbeach","locality":"Spalding","postal_code":"PE12 9PB","premises":"Norfolk House Farm","region":"Lincolnshire"},"country_of_residence":"England","date_of_birth":{"month":2,"year":1971},"etag":"401ca7e16bf5effef24fd2abf9c0da723b84a74f","kind":"individual-person-with-significant-control","links":{"self":"/company/04333110/persons-with-significant-control/individual/KTvVTJijcv8RGXnU3Cub8GBf0Gc"},"name":"Mr Stuart William Piccaver","name_elements":{"forename":"Stuart","middle_name":"William","surname":"Piccaver","title":"Mr"},"nationality":"British","natures_of_control":["ownership-of-shares-25-to-50-percent","voting-rights-25-to-50-percent"],"notified_on":"2016-04-06"}} 

In [10]:
import pandas as pd

# Single dict -> one-row DataFrame with flattened columns
flat = pd.json_normalize(df2, sep='.')
print(flat.columns.tolist())   # see all columns
display(flat.T)                # view all fields vertically

['company_number', 'data.address.address_line_1', 'data.address.address_line_2', 'data.address.locality', 'data.address.postal_code', 'data.address.premises', 'data.address.region', 'data.country_of_residence', 'data.date_of_birth.month', 'data.date_of_birth.year', 'data.etag', 'data.kind', 'data.links.self', 'data.name', 'data.name_elements.forename', 'data.name_elements.middle_name', 'data.name_elements.surname', 'data.name_elements.title', 'data.nationality', 'data.natures_of_control', 'data.notified_on']


,0
company_number,04333110
data.address.address_line_1,Gedney Marsh
data.address.address_line_2,Holbeach
data.address.locality,Spalding
data.address.postal_code,PE12 9PB
data.address.premises,Norfolk House Farm
data.address.region,Lincolnshire
data.country_of_residence,England
data.date_of_birth.month,2
data.date_of_birth.year,1971


In [12]:
data_only = pd.json_normalize(df2['data'], sep='.')
data_only['company_number'] = df2['company_number']
display(data_only.T)

,0
country_of_residence,England
etag,401ca7e16bf5effef24fd2abf9c0da723b84a74f
kind,individual-person-with-significant-control
name,Mr Stuart William Piccaver
nationality,British
natures_of_control,"[ownership-of-shares-25-to-50-percent, voting-..."
notified_on,2016-04-06
address.address_line_1,Gedney Marsh
address.address_line_2,Holbeach
address.locality,Spalding


In [14]:
rows = [df2, ...]  # list of dicts
flat = pd.json_normalize(rows, sep='.')
print(flat.columns.tolist())

['company_number', 'data.address.address_line_1', 'data.address.address_line_2', 'data.address.locality', 'data.address.postal_code', 'data.address.premises', 'data.address.region', 'data.country_of_residence', 'data.date_of_birth.month', 'data.date_of_birth.year', 'data.etag', 'data.kind', 'data.links.self', 'data.name', 'data.name_elements.forename', 'data.name_elements.middle_name', 'data.name_elements.surname', 'data.name_elements.title', 'data.nationality', 'data.natures_of_control', 'data.notified_on']


In [15]:
import json
import pandas as pd

psc_path = "/home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc-snapshot-2025-10-14_1of30 (1)/psc-snapshot-2025-10-14_1of30.txt"

try:
    # NDJSON fast path
    chunk = next(pd.read_json(psc_path, lines=True, chunksize=500))
    flat = pd.json_normalize(chunk.to_dict(orient='records'), sep='.')
except ValueError:
    # Fallback: TSV/CSV with a JSON 'data' column
    tsv = pd.read_csv(psc_path, sep="\t", nrows=500)
    if 'data' in tsv.columns:
        parsed = []
        for x in tsv['data']:
            if isinstance(x, str):
                try:
                    parsed.append(json.loads(x))
                except Exception:
                    parsed.append({})
            elif isinstance(x, dict):
                parsed.append(x)
            else:
                parsed.append({})
        flat = pd.json_normalize(parsed, sep='.')
        if 'company_number' in tsv.columns and 'company_number' not in flat.columns:
            flat['company_number'] = tsv['company_number'].values[:len(flat)]
    else:
        flat = tsv  # best-effort

display(flat.head())
print(f"Rows: {len(flat)}, Columns: {len(flat.columns)}")

,company_number,data.address.address_line_1,data.address.address_line_2,data.address.country,data.address.locality,data.address.postal_code,data.address.premises,data.etag,data.identification.country_registered,data.identification.legal_authority,data.identification.legal_form,data.identification.place_registered,data.identification.registration_number,data.kind,data.links.self,data.name,data.natures_of_control,data.notified_on,data.country_of_residence,data.date_of_birth.month,data.date_of_birth.year,data.name_elements.forename,data.name_elements.middle_name,data.name_elements.surname,data.name_elements.title,data.nationality,data.address.region,data.ceased_on,data.address.care_of,data.address.po_box
0,07434180,Boddington Lane,Boddington,England,Cheltenham,GL51 0TJ,The Manor,706b6e8773700f85361929bb5cd0214d630de6e0,England & Wales,Companies Act,Private Limited Company,Companies House Cardiff,00686734,corporate-entity-person-with-significant-control,/company/07434180/persons-with-significant-con...,Robert Hitchins Limited,"[ownership-of-shares-75-to-100-percent, voting...",2016-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08810260,Godmans Lane,Kirk Ella,NaN,Hull,HU10 7NY,Alvenga,224fe8afa8a6ff381ea0069dac5c6d4c995f15b2,NaN,NaN,NaN,NaN,NaN,individual-person-with-significant-control,/company/08810260/persons-with-significant-con...,Mr Michael Charles Saunders,"[ownership-of-shares-75-to-100-percent, voting...",2016-04-06,England,8.0,1951.0,Michael,Charles,Saunders,Mr,British,NaN,NaN,NaN,NaN
2,06022970,20 Rugby Road Milverton,Leamington Spa,NaN,Warwickshire,CV32 6DG,NaN,1840e605a8948c04246e8d9904da789e2fdc4a7e,NaN,NaN,NaN,NaN,NaN,individual-person-with-significant-control,/company/06022970/persons-with-significant-con...,Mrs Jean Carol Randle,[ownership-of-shares-50-to-75-percent],2016-04-06,England,4.0,1945.0,Jean,Carol,Randle,Mrs,British,NaN,NaN,NaN,NaN
3,08301480,Station Road,NaN,England,Upminster,RM14 2SJ,"Suite 17, Essex House",0705b53fe3b34355fe183ae68013e8e1e19fda42,NaN,NaN,NaN,NaN,NaN,individual-person-with-significant-control,/company/08301480/persons-with-significant-con...,Mr Robert Power,[right-to-appoint-and-remove-directors],2016-04-06,England,1.0,1983.0,Robert,NaN,Power,Mr,British,Essex,NaN,NaN,NaN
4,07428111,Cottage,Ardens Grafton,NaN,Alcester,B49 6DS,Malthouse,3ea7a8ff70ce98944c276648a875257e9b127428,NaN,NaN,NaN,NaN,NaN,individual-person-with-significant-control,/company/07428111/persons-with-significant-con...,Mr Andrew John Cain,[ownership-of-shares-75-to-100-percent],2016-04-06,England,11.0,1964.0,Andrew,John,Cain,Mr,British,Warwickshire,NaN,NaN,NaN


Rows: 500, Columns: 30


In [16]:
out_path = "/home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc_flat_500.csv"
flat.to_csv(out_path, index=False)
print(f"Saved {len(flat)} rows x {len(flat.columns)} cols to {out_path}")

Saved 500 rows x 30 cols to /home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc_flat_500.csv
